# Lesson 5: Excitatory synaptic integration

## Lesson goals:

1. Discuss papers from last week focusing on the role of NMDA-type glutamate receptors in spatial and temporal input summation
2. Discuss challenge from last week: what happens to spike backpropagation in the distal dendrite if an EPSP precedes the spike by ~5 ms?
3. What happened to somatic excitability after manually tuning a dendritic compartment?
4. How to activate postsynaptic currents with presynaptic spikes in NEURON: NetCon and VecStim
5. Emulate glutamate uncaging experiment and quantify synaptic integration.
6. Adding NMDA-type glutamate receptors to acheive supralinear input summation.
7. Papers for next week focusing on heterogeneity of interneuron cell types and their impact on dendritic integration.

## 1. Discuss papers from last week focusing on the role of NMDA-type glutamate receptors in spatial and temporal input summation

- Losonczy, A., Magee, J. (2006). Integrative Properties of Radial Oblique Dendrites in Hippocampal CA1 Pyramidal Neurons Neuron  50(2), 291-307. https://dx.doi.org/10.1016/j.neuron.2006.03.016
- Popescu, G., Robert, A., Howe, J., Auerbach, A. (2004). Reaction mechanism determines NMDA receptor response to repetitive stimulation Nature  430(7001), 790-793. https://dx.doi.org/10.1038/nature02775

## 2. Discuss challenge from last week: what happens to spike backpropagation in the distal dendrite if an EPSP precedes the spike by ~5 ms?

## 3. What happened to somatic excitability after manually tuning a dendritic compartment?

In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from scipy.stats import linregress
import math
from neuron import h
h.load_file('stdrun.hoc')

### Let's start with our multi-compartment neuron model from last week:

In [ ]:
# We're starting to accumulate a lot of helper functions necessary to specify, simulate, and analyze our simulations.
# Soon it will be sensible to organize our code into a proper python module with classes and functions that can be conveniently imported


def lambda_f(sec, f=100.):
    """
    Calculates the AC length constant for the given section at the frequency f.
    :param sec: :class:'h.Section'
    :param f: float (Hz)
    :return: float (um)
    """
    diam = np.mean([seg.diam for seg in sec])
    Ra = sec.Ra
    cm = np.mean([seg.cm for seg in sec])
    return 1e5 * math.sqrt(diam / (4. * math.pi * f * Ra * cm))


def get_lambda_nseg(sec, lam=0.1, f=100.):
    """
    Used to determine the number of segments per hoc section to achieve the desired spatial and temporal resolution.
    Calculates the AC length constant for the given section at the frequency f.
    Assures that no segment will be larger than a fraction of the length_constant, specified by the proportion lam.
    :param sec : :class:'h.Section'
    :param lam : float: no segment will be larger than lam, a fraction of the section's length constant 
    :param f: float (Hz)
    :return : int
    """
    length_constant = lambda_f(sec, f)
    return int(((sec.L / (lam * length_constant)) + 0.9) / 2) * 2 + 1


def get_event_peak_amp(t, v, baseline_window=(195., 200.),  measurement_window=(200., 205.), direction=1.):
    baseline_indexes = np.where((baseline_window[0] <= t) & (t < baseline_window[1]))
    measurement_indexes = np.where((measurement_window[0] <= t) & (t < measurement_window[1]))
    baseline_v = np.mean(v[baseline_indexes])
    if direction > 0.:
        measurement_v = np.max(v[measurement_indexes])
    else:
        measurement_v = np.min(v[measurement_indexes])
    delta_v = measurement_v - baseline_v
    return delta_v


def update_model(x):
    # parse the input parameters
    gnabar, gkdrbar, gkabar, gl = x
    
    soma(0.5).nax.gbar = gnabar
    soma(0.5).kdr.gkdrbar = gkdrbar
    soma(0.5).kap.gkabar = gkabar
    soma(0.5).pas.g = gl

In [ ]:
soma = h.Section()

# Set Ra to a physiological value
Ra0 = 150.  # Ohm-cm
soma.Ra = Ra0
soma.L = 20.
soma.diam = 20.
soma.insert('pas')  # leak
soma.insert('kdr')  # delayed-rectifier K+ channel
soma.insert('kap')  # A-type K+ channel
soma.insert('nax')  # Na+ channel

soma(0.5).pas.e = -65.
gl0 = soma(0.5).pas.g
gkdrbar0 = soma(0.5).kdr.gkdrbar
gkabar0 = soma(0.5).kap.gkabar
gnabar0 = soma(0.5).nax.gbar

# obtained via coarse search with basinhopping followed by local polishing with simplex
best_x = soma_gnabar, soma_gkdrbar, soma_gkabar, soma_gl = [0.09389553, 0.02495658, 0.0331321,  0.00052055]

update_model(best_x)

dend = h.Section()
dend.L = 300.
dend.diam = 2.
# Set Ra to the same value as the soma, it's a property of the cytoplasm
dend.Ra = Ra0
dend.nseg = get_lambda_nseg(dend)
# parent section, parent location, child location
dend.connect(soma, 1., 0.)

dend.insert('pas')
dend.insert('nax')
dend.insert('kdr')
dend.insert('kap')

ka_gradient = lambda distance, slope: soma_gkabar + slope * distance
ka_slope = 1.5 * soma_gkabar / 100.  # increase the somatic conductance density by 1.5 * the somatic value every 100 um

# Last week, we manually determined:
# 1. A value for dendritic leak conductance that produced realistic soma-dendrite attenutation.
# 2. Values for dendritic Na+, DR-type K+, and A-type K+ conductance that produced realistic AP backpropagation
# 3. The slope of a linear gradient of A-type K+ conductance in dendrites that increases with distance from the soma
for seg in dend:
    seg.pas.e = -65.
    seg.pas.g = 1.5 * soma_gl
    seg.nax.gbar = 0.1 * soma_gnabar
    seg.kdr.gkdrbar = soma_gkdrbar
    distance = seg.x * dend.L
    seg.kap.gkabar = ka_gradient(distance, ka_slope)

h.celsius = 35.  # These mechanisms are meant to work at (closer to) physiological temperature
h.tstop = 600.
v_init = -65.
h.v_init = v_init

t = h.Vector()
soma_voltage = h.Vector()
t.record(h._ref_t, h.dt)  # record the time base
soma_voltage.record(soma(0.5)._ref_v, h.dt)  # record the voltage across the membrane in a segment

step_current_stim = h.IClamp(soma(0.5))
f_I_stim_dur = 200.
f_I_stim_delay = 200.
f_I_stim_amp_list = [0.05 * i for i in range(10)]

R_inp_stim_dur = 200.
R_inp_stim_delay = 200.
R_inp_stim_amp = -0.1

stim_params = {
    'f_I': {'delay': f_I_stim_delay, 'dur': f_I_stim_dur, 'amp_list': f_I_stim_amp_list},
    'R_inp': {'delay': R_inp_stim_delay, 'dur': R_inp_stim_dur, 'amp': R_inp_stim_amp}
              }

spike_times = h.Vector()
spike_detector = h.NetCon(soma(0.5)._ref_v, None, sec=soma)
spike_detector.delay = 0.  # ms
spike_detector.threshold = -10.  # mV
spike_detector.record(spike_times)

### How did adding the dendrite compartment affect excitability at the soma?

In [ ]:
sim_history = defaultdict(lambda: defaultdict(dict))

bounds = [(0.1*xi, 10.*xi) for xi in best_x]

target = {}
target['f_I_slope'] = 200.  # Hz / nA
target['soma_R_inp'] = 150.  # MOhm


fit_line = lambda x, slope, intercept: x * slope + intercept


def simulate_f_I(stim_params):
    sim_summary_list = []
    step_current_stim.delay = stim_params['f_I']['delay']  # start time of current injection
    step_current_stim.dur = stim_params['f_I']['dur']  # duration in milliseconds
    for stim_amp in stim_params['f_I']['amp_list']:
        this_sim_summary = {}
        step_current_stim.amp = stim_amp
        this_sim_summary['stim_amp'] = stim_amp
        this_sim_summary['stim_delay'] = stim_params['f_I']['delay']
        this_sim_summary['stim_dur'] = stim_params['f_I']['dur']
        h.run()
        this_sim_summary['t'] = np.array(t)
        this_sim_summary['soma_voltage'] = np.array(soma_voltage)
        this_spike_times = np.array(spike_times)
        this_sim_summary['spike_times'] = this_spike_times
        
        # take only spike times occurring inside the current injection
        valid_spike_indexes = np.where((this_spike_times > f_I_stim_delay) & (this_spike_times <= f_I_stim_delay + f_I_stim_dur))[0]
        this_sim_summary['spike_count'] = len(valid_spike_indexes)
        this_sim_summary['spike_rate'] = len(valid_spike_indexes) / (f_I_stim_dur / 1000.)  # convert ms to s
        sim_summary_list.append(this_sim_summary)
    step_current_stim.amp = 0.
    
    return sim_summary_list


def fit_f_I_curve(stim_amp, spike_rate):
    stim_amp_array = np.array(stim_amp)
    spike_rate_array = np.array(spike_rate)
    valid_stim_indexes = np.where(spike_rate_array > 0.)[0]
    if len(valid_stim_indexes) < 2:
        return 0., 0.
    slope, intercept, r_value, p_value, std_err = linregress(stim_amp_array[valid_stim_indexes], spike_rate_array[valid_stim_indexes])
    
    return slope, intercept


def get_f_I_slope_error(x, target, bounds, sim_history, stim_params):
    # enforce bounds:
    for xi, bi in zip(x, bounds):
        if not bi[0] <= xi <= bi[1]:
            return 1e9
    
    update_model(x)
    
    # simulate f_I and retrieve the data
    this_f_I_sim_summary_list = simulate_f_I(stim_params)
    
    # calculate the f_I slope
    this_spike_rate_list = [this_f_I_sim_summary['spike_rate'] for this_f_I_sim_summary in this_f_I_sim_summary_list]
    this_stim_amp_list = stim_params['f_I']['amp_list']
    this_f_I_slope, this_f_I_intercept = fit_f_I_curve(this_stim_amp_list, this_spike_rate_list)
    
    # calculate the squared error
    this_f_I_error = (this_f_I_slope - target['f_I_slope']) ** 2.
    
    sim_history[tuple(x)]['f_I']['data'] = this_f_I_sim_summary_list
    sim_history[tuple(x)]['f_I']['slope'] = this_f_I_slope
    sim_history[tuple(x)]['f_I']['intercept'] = this_f_I_intercept
    sim_history[tuple(x)]['f_I']['error'] = this_f_I_error
    
    return this_f_I_error


def calculate_R_inp(t, v, i_amp, baseline_window=(195., 200.),  measurement_window=(395., 400.)):
    baseline_indexes = np.where((baseline_window[0] <= t) & (t < baseline_window[1]))
    measurement_indexes = np.where((measurement_window[0] <= t) & (t < measurement_window[1]))
    baseline_v = np.mean(v[baseline_indexes])
    measurement_v = np.mean(v[measurement_indexes])
    delta_v = abs(baseline_v - measurement_v)
    R_inp = (delta_v / 1000.) / (abs(i_amp) / 1e9) / 1e6 # convert mV to V, nA to A, and Ohm to MegaOhm
    return R_inp


def simulate_soma_R_inp(stim_params):
    step_current_stim.delay = stim_params['R_inp']['delay']
    step_current_stim.dur = stim_params['R_inp']['dur']
    step_current_stim.amp = stim_params['R_inp']['amp']
    this_sim_summary = {}
    this_sim_summary['stim_delay'] = stim_params['R_inp']['delay']
    this_sim_summary['stim_dur'] = stim_params['R_inp']['dur']
    this_sim_summary['stim_amp'] = stim_params['R_inp']['amp']
    
    h.run()
    t_array = np.array(t)
    v_array = np.array(soma_voltage)
    soma_R_inp = calculate_R_inp(t_array, v_array, stim_params['R_inp']['amp'])
    
    this_sim_summary['t'] = t_array
    this_sim_summary['soma_voltage'] = v_array
    this_sim_summary['soma_R_inp'] = soma_R_inp
    step_current_stim.amp = 0.
    
    return [this_sim_summary]  # for consistency with f_I functions, return a list of simulation summary dictionaries


def get_soma_R_inp_error(x, target, bounds, sim_history, stim_params):
    # enforce bounds
    for xi, bi in zip(x, bounds):
        if not bi[0] <= xi <= bi[1]:
            return 1e9
        
    update_model(x)
    
    this_R_inp_sim_summary_list = simulate_soma_R_inp(stim_params)
    this_soma_R_inp = this_R_inp_sim_summary_list[0]['soma_R_inp']
    
    # calculate the squared error
    this_R_inp_error = (this_soma_R_inp - target['soma_R_inp']) ** 2.
    
    sim_history[tuple(x)]['R_inp']['data'] = this_R_inp_sim_summary_list
    sim_history[tuple(x)]['R_inp']['soma_R_inp'] = this_soma_R_inp
    sim_history[tuple(x)]['R_inp']['error'] = this_R_inp_error
    
    return this_R_inp_error


def get_excitability_error(x, target, bounds, sim_history, stim_params):
    this_R_inp_error = get_soma_R_inp_error(x, target, bounds, sim_history, stim_params)
    this_f_I_slope_error = get_f_I_slope_error(x, target, bounds, sim_history, stim_params)
    
    return this_R_inp_error + this_f_I_slope_error

In [ ]:
error = get_excitability_error(best_x, target, bounds, sim_history, stim_params)
print('x:', best_x, 'error:', error)

In [ ]:
fig, axis = plt.subplots()
for i, sim_summary in enumerate(sim_history[tuple(best_x)]['R_inp']['data']):
    axis.plot(sim_summary['t'], sim_summary['soma_voltage'])
    axis.set_ylabel('Voltage (mV)')
    axis.set_xlabel('Time (ms)')
fig.tight_layout()

print('soma_R_inp:', sim_history[tuple(best_x)]['R_inp']['soma_R_inp'], 'Error:', sim_history[tuple(best_x)]['R_inp']['error'])

fig, axes = plt.subplots(3, 4, figsize=(12., 8.))
for i, sim_summary in enumerate(sim_history[tuple(best_x)]['f_I']['data']):
    axes.flat[i].plot(sim_summary['t'], sim_summary['soma_voltage'])
    axes.flat[i].set_ylabel('Voltage (mV)')
    axes.flat[i].set_xlabel('Time (ms)')
fig.tight_layout()

this_slope = sim_history[tuple(best_x)]['f_I']['slope']
this_intercept = sim_history[tuple(best_x)]['f_I']['intercept']
stim_amp_array = np.array([sim_summary['stim_amp'] for sim_summary in sim_history[tuple(best_x)]['f_I']['data']])
spike_rate_array = np.array([sim_summary['spike_rate'] for sim_summary in sim_history[tuple(best_x)]['f_I']['data']])
valid_stim_indexes = np.where(spike_rate_array > 0.)[0]

plt.figure()
plt.scatter(stim_amp_array, spike_rate_array)
plt.plot(stim_amp_array[valid_stim_indexes], fit_line(stim_amp_array[valid_stim_indexes], this_slope, this_intercept))
plt.ylabel('Firing rate (Hz)')
plt.xlabel('Current amplitude (nA)')
plt.show()

print('f-I slope:', sim_history[tuple(best_x)]['f_I']['slope'], 'Error:', sim_history[tuple(best_x)]['f_I']['error'])

### So we would have to simultaneously optimize both the somatic and dendritic parameters to get that right.

## 4. How to activate postsynaptic currents with presynaptic spikes in NEURON: NetCon and VecStim

Last week we manually constructed an EPSC-shaped current waveform to inject into the dendrite. What if we want to be able to trigger EPSCs with spikes?

Let's use a synaptic point process that generates a current with exponential rise and decay whenever it is triggered by a spike source: <br>
Take a look at the source code in exp2EPSC.mod, and don't forget to complile this new mechanism with `nrnivmodl`

In [ ]:
# Let's insert a synapse in the dendrite ~150 um from the soma:
syn = h.EPSC(dend(0.75))

In [ ]:
# tab to complete to explore attributes of the synaptic point process object:
syn.

We can control the kinetics of this current with the `tau_rise` and `tau_decay` attributes:

In [ ]:
syn.tau_rise = 0.5
syn.tau_decay = 10.

The `h.NetCon` object is used to deliver spikes detected in a presynaptic spike detector to a postsynaptic target point process. <br>
But in this case we don't have a presynaptic cell, we just want to manually define a vector of spike times. <br>
For this we need a special "Artificial Cell" called a `VecStim`

In [ ]:
vs = h.VecStim()

In [ ]:
# tab to complete to explore attributes of the VecStim object:
vs.

# The play() method can be used to load a vector of spike times

Now we can connect the VecStim to our synaptic point process using a `NetCon`:

In [ ]:
nc = h.NetCon(vs, syn)

In [ ]:
# tab to complete to explore attributes of the NetCon object:
nc.

What are the default values of the `delay` and `weight` attributes?

In [ ]:
print('Delay:', nc.delay, 'Weight', nc.weight)

The `delay` attribute is used to emulate axonal conduction delays. For now, we want our spikes to activate synapses without delay, so this can be set to zero.

In [ ]:
nc.delay = 0.

The `weight` attribute of a `NetCon` is actually a vector. For the EPSC mechanism, just the first element in this vector is used, and it controls the amplitude of the postsynaptic current:

In [ ]:
print('Delay:', nc.delay, 'Weight', nc.weight[0])

Let's set the `weight` attribute to be one for now (the synaptic amplitude is also controlled by the `i_unit` attribute of the EPSC object).

In [ ]:
nc.weight[0] = 1.

Now we need a dendritic recording, and a spike train to play into our `VecStim` --> `NetCon` --> `EPSC` object chain. <br>
We can also record the actual current through the synaptic point process:

In [ ]:
dend_voltage = h.Vector()
dend_voltage.record(dend(0.75)._ref_v, h.dt)

presyn_spike_times_list = [250.]
vs.play(h.Vector(presyn_spike_times_list))

syn_current = h.Vector()
syn_current.record(syn._ref_i, h.dt)

### Run a simulation and plot the results:

In [ ]:
h.run()

# It's good practice to play an empty vector in to the `VecStim` at the end of every simulation to "reset" it.
vs.play(h.Vector())

In [ ]:
fig, axes = plt.subplots(2, figsize=(6., 6.))
axes[0].plot(t, soma_voltage, label='Soma Vm')
axes[0].plot(t, dend_voltage, label='Dend Vm')
axes[0].set_ylabel('Voltage (mV)')
axes[0].set_xlabel('Time (ms)')
axes[1].plot(t, syn_current)
axes[1].set_ylabel('Current (nA)')
axes[1].set_xlabel('Time (ms)')
axes[0].legend(loc='best', frameon=False, framealpha=0.5)
fig.tight_layout()
fig.show()

That was way too much current! It actually kicked off a dendritic sodium spike, followed by a full-blown somatic action potential!

In [ ]:
syn.i_unit = -0.01
vs.play(h.Vector(presyn_spike_times_list))
h.run()
vs.play(h.Vector())

In [ ]:
fig, axes = plt.subplots(2, figsize=(6., 6.))
axes[0].plot(t, soma_voltage, label='Soma Vm')
axes[0].plot(t, dend_voltage, label='Dend Vm')
axes[0].set_ylabel('Voltage (mV)')
axes[0].set_xlabel('Time (ms)')
axes[1].plot(t, syn_current)
axes[1].set_ylabel('Current (nA)')
axes[1].set_xlabel('Time (ms)')
axes[0].legend(loc='best', frameon=False, framealpha=0.5)
fig.tight_layout()
fig.show()

This is better. ~0.2 mV EPSP at the soma is more realistic.

### What happens if we stimulate this single synapse repetitively in a 100 Hz train?

In [ ]:
presyn_spike_times_list = [250. + 10. * i for i in range(5)]
vs.play(h.Vector(presyn_spike_times_list))
h.run()
vs.play(h.Vector())

In [ ]:
fig, axes = plt.subplots(2, figsize=(6., 6.))
axes[0].plot(t, soma_voltage, label='Soma Vm')
axes[0].plot(t, dend_voltage, label='Dend Vm')
axes[0].set_ylabel('Voltage (mV)')
axes[0].set_xlabel('Time (ms)')
axes[1].plot(t, syn_current)
axes[1].set_ylabel('Current (nA)')
axes[1].set_xlabel('Time (ms)')
axes[0].legend(loc='best', frameon=False, framealpha=0.5)
fig.tight_layout()
fig.show()

### What is unrealistic about this?

First of all, synaptic currents are generated by ion channels and are subject to ionic driving force, so successive events at increasing dendritic voltage should open the same peak conductance, but with decreasing driving force, and so decreasing current amplitude...

There's another point process we can use that activates a true conductance rather than an idealized current: Exp2EPSG

In [ ]:
syn = h.Exp2EPSG(dend(0.75))

In [ ]:
# tab to complete to explore attributes of the synaptic point process object:
syn.

For this mechanism, `tau1` is the rise time constant, and `tau2` is the decay time constant. <br>
`gmax` is the peak conductance, and `e` is the reversal potential.

In [ ]:
syn.tau1 = 0.5  # ms
syn.tau2 = 10.  # ms
syn.gmax = 0.01  # uS
syn.e = 0.  # mV

This new point process will have to be connected to the `VecStim` via a `NetCon` again:

In [ ]:
vs = h.VecStim()

nc = h.NetCon(vs, syn)
nc.delay = 0.
nc.weight[0] = 1.

We can also now establish recordings of both `i` and `g` from the new synapse:

In [ ]:
syn_current = h.Vector()
syn_current.record(syn._ref_i, h.dt)

syn_conductance = h.Vector()
syn_conductance.record(syn._ref_g, h.dt)

Let's first stimulate the synapse with a single pulse:

In [ ]:
presyn_spike_times_list = [250.]
vs.play(h.Vector(presyn_spike_times_list))
h.run()
vs.play(h.Vector())

In [ ]:
fig, axes = plt.subplots(3, figsize=(6., 9.))
axes[0].plot(t, soma_voltage, label='Soma Vm')
axes[0].plot(t, dend_voltage, label='Dend Vm')
axes[0].set_ylabel('Voltage (mV)')
axes[0].set_xlabel('Time (ms)')
axes[1].plot(t, syn_conductance)
axes[1].set_ylabel('Normalized conductance')
axes[1].set_xlabel('Time (ms)')
axes[2].plot(t, syn_current)
axes[2].set_ylabel('Current (nA)')
axes[2].set_xlabel('Time (ms)')
axes[0].legend(loc='best', frameon=False, framealpha=0.5)
fig.tight_layout()
fig.show()

Look's like we'll have to use a much lower value of gmax to produce ~0.01 nA EPSC and ~0.2 mV somatic EPSP

In [ ]:
syn.gmax = 0.0002

In [ ]:
presyn_spike_times_list = [250.]
vs.play(h.Vector(presyn_spike_times_list))
h.run()
vs.play(h.Vector())

In [ ]:
fig, axes = plt.subplots(3, figsize=(6., 9.))
axes[0].plot(t, soma_voltage, label='Soma Vm')
axes[0].plot(t, dend_voltage, label='Dend Vm')
axes[0].set_ylabel('Voltage (mV)')
axes[0].set_xlabel('Time (ms)')
axes[1].plot(t, syn_conductance)
axes[1].set_ylabel('Normalized conductance')
axes[1].set_xlabel('Time (ms)')
axes[2].plot(t, syn_current)
axes[2].set_ylabel('Current (nA)')
axes[2].set_xlabel('Time (ms)')
axes[0].legend(loc='best', frameon=False, framealpha=0.5)
fig.tight_layout()
fig.show()

### Now let's try stimulating in a train again:

In [ ]:
presyn_spike_times_list = [250. + 10. * i for i in range(5)]
vs.play(h.Vector(presyn_spike_times_list))
h.run()
vs.play(h.Vector())

In [ ]:
fig, axes = plt.subplots(3, figsize=(6., 9.))
axes[0].plot(t, soma_voltage, label='Soma Vm')
axes[0].plot(t, dend_voltage, label='Dend Vm')
axes[0].set_ylabel('Voltage (mV)')
axes[0].set_xlabel('Time (ms)')
axes[1].plot(t, syn_conductance)
axes[1].set_ylabel('Normalized conductance')
axes[1].set_xlabel('Time (ms)')
axes[2].plot(t, syn_current)
axes[2].set_ylabel('Current (nA)')
axes[2].set_xlabel('Time (ms)')
axes[0].legend(loc='best', frameon=False, framealpha=0.5)
fig.tight_layout()
fig.show()

### Now why is the conductance increasing apparently without bound?

To emulate synaptic AMPA-type glutamate receptors, that are saturated on a single pulse, we need a mechanism that accounts for this! <br>
Now let's try the synaptic point process SatExp2Syn:

In [ ]:
syn = h.SatExp2Syn(dend(0.75))

In [ ]:
# tab to complete to explore attributes of the synaptic point process object:
syn.

This mechanism is again parameterized differently. `dur_onset` controls the rise kinetics, `tau_offset` controls the decay kinetics, `e` controls the reversal potential, but now the unitary single-pulse peak conductance is set by the second element in the `NetCon.weight` vector.

In [ ]:
syn.dur_onset = 1.
syn.tau_offset = 10.
syn.e = 0.

vs = h.VecStim()
nc = h.NetCon(vs, syn)
nc.delay = 0.
nc.weight[0] = 1.
nc.weight[1] = 0.0002

Now there is also a `sat` attribute that controls how close to saturation a single pulse should drive the receptor.

In [ ]:
syn.sat = 0.99

Don't forget to re-establish recordings from the new synapse:

In [ ]:
syn_current = h.Vector()
syn_current.record(syn._ref_i, h.dt)

syn_conductance = h.Vector()
syn_conductance.record(syn._ref_g, h.dt)

In [ ]:
presyn_spike_times_list = [250.]
vs.play(h.Vector(presyn_spike_times_list))
h.run()
vs.play(h.Vector())

In [ ]:
fig, axes = plt.subplots(3, figsize=(6., 9.))
axes[0].plot(t, soma_voltage, label='Soma Vm')
axes[0].plot(t, dend_voltage, label='Dend Vm')
axes[0].set_ylabel('Voltage (mV)')
axes[0].set_xlabel('Time (ms)')
axes[1].plot(t, syn_conductance)
axes[1].set_ylabel('Conductance (uS)')
axes[1].set_xlabel('Time (ms)')
axes[2].plot(t, syn_current)
axes[2].set_ylabel('Current (nA)')
axes[2].set_xlabel('Time (ms)')
axes[0].legend(loc='best', frameon=False, framealpha=0.5)
fig.tight_layout()
fig.show()

### Now the train:

In [ ]:
presyn_spike_times_list = [250. + 10. * i for i in range(5)]
vs.play(h.Vector(presyn_spike_times_list))
h.run()
vs.play(h.Vector())

In [ ]:
fig, axes = plt.subplots(3, figsize=(6., 9.))
axes[0].plot(t, soma_voltage, label='Soma Vm')
axes[0].plot(t, dend_voltage, label='Dend Vm')
axes[0].set_ylabel('Voltage (mV)')
axes[0].set_xlabel('Time (ms)')
axes[1].plot(t, syn_conductance)
axes[1].set_ylabel('Conductance (uS)')
axes[1].set_xlabel('Time (ms)')
axes[2].plot(t, syn_current)
axes[2].set_ylabel('Current (nA)')
axes[2].set_xlabel('Time (ms)')
axes[0].legend(loc='best', frameon=False, framealpha=0.5)
fig.tight_layout()
fig.show()

### Great, now we have saturating, conductance-based AMPA synapses. <br>

## 5. Emulate glutamate uncaging experiment and quantify synaptic integration.

Now let's try to reproduce the synaptic integration experiment from Losonczy & Magee, 2006. <br>
First, we need a set of ~20 synapses within a small section of dendrite:

In [ ]:
syn_list = []
vs_list = []
nc_list = []

for i in range(20):
    syn = h.SatExp2Syn(dend(0.75))
    syn.dur_onset = 1.
    syn.tau_offset = 10.
    syn.e = 0.
    syn.sat = 0.99
    syn_list.append(syn)
    
    vs = h.VecStim()
    vs_list.append(vs)
    
    nc = h.NetCon(vs, syn)
    nc.delay = 0.
    nc.weight[0] = 1.
    nc.weight[1] = 0.0002
    nc_list.append(nc)

First let's run a set of simulations where we only stimulate one synapse at a time, and save the unitary responses:

In [ ]:
presyn_spike_times_list = [250.]
raw_soma_unitary_EPSP_list = []
for i in range(20):
    vs_list[i].play(h.Vector(presyn_spike_times_list))
    h.run()
    vs_list[i].play(h.Vector())
    # save an array of the time base
    if i == 0:
        t_array = np.array(t)
    soma_voltage_array = np.array(soma_voltage)
    raw_soma_unitary_EPSP_list.append(soma_voltage_array)
    vs_list[i].play(h.Vector())

In [ ]:
# plot the unitary responses:
plt.figure()
for i in range(20):
    plt.plot(t_array, raw_soma_unitary_EPSP_list[i])

Now we need a way to compute the arithmetic sum of these unitary EPSPs given an arbitrary inter-stimulus interval:

In [ ]:
def offset_and_truncate_trace_list(trace_list, t, start, window_dur=200., baseline_dur=10.):
    """
    Given a list of traces and a reference time base, subtract the mean baseline value and truncate.
    :param trace_list: list of array
    :param t: array
    :param start: float
    :param window_dur: float
    :param baseline_dur: float
    :return: tuple: (list of array, array)
    """
    t_indexes = np.where((t >= start - baseline_dur) & (t < start + window_dur))[0]
    truncated_t = t[t_indexes] - start
    baseline_indexes = np.where(truncated_t < 0.)[0]
    truncated_traces = []
    for i in range(len(trace_list)):
        this_truncated_trace = trace_list[i][t_indexes]
        baseline = np.mean(this_truncated_trace[baseline_indexes])
        this_truncated_trace -= baseline
        truncated_traces.append(this_truncated_trace)
    
    return truncated_traces, truncated_t


def get_expected_compound_EPSP_list(unitary_EPSP_list, t, ISI):
    """
    Given a list of unitary EPSP amplitude traces, produce a list of compound EPSP traces with increasing numbers of synapses linearly summed.
    :param unitary_EPSP_list: list of array
    :param t: array
    :param ISI: float
    :return: list of array
    """
    dt = t[1] - t[0]
    ISI_len = int(ISI / dt)
    window_len = len(t)
    
    exp_compound_EPSP_list = []
    current_compound_EPSP = np.zeros_like(t)
    this_t_offset = 0
    for i in range(len(unitary_EPSP_list)):
        if window_len < this_t_offset:
            raise RuntimeError('Window too short to sum %i units with ISI=%.2f ms' % (i + 1, ISI))
        this_unit = unitary_EPSP_list[i]
        current_compound_EPSP[this_t_offset:window_len] += this_unit[:window_len - this_t_offset]
        exp_compound_EPSP_list.append(np.copy(current_compound_EPSP))
        this_t_offset += ISI_len
    
    return exp_compound_EPSP_list

In Losonczy & Magee, 2006, ISI=0.1 ms was actually ISI=0.3 ms including laser dwell time + laser move time

In [ ]:
# First let's baseline subtract and truncate these traces so that we can use them for offline summing:
truncated_soma_unitary_EPSP_list, truncated_t = offset_and_truncate_trace_list(raw_soma_unitary_EPSP_list, t_array, start=250.)

# Now we will build a list of traces with increasing numbers of units summed:
expected_compound_EPSP_list = get_expected_compound_EPSP_list(truncated_soma_unitary_EPSP_list, truncated_t, ISI=0.3)

In [ ]:
plt.figure()
for i in range(len(expected_compound_EPSP_list)):
    plt.plot(truncated_t, expected_compound_EPSP_list[i])
    plt.ylabel('Expected EPSP Amplitude (mV)')
    plt.xlabel('Time (ms)')

Now, we can run a set of simulations where we increase the number of stimulated inputs with the specified ISI:

In [ ]:
raw_soma_voltage_list = []

ISI = 0.3
t_offset = 0.
for i in range(20):
    presyn_spike_times_list = [250. + i * ISI]
    vs_list[i].play(h.Vector(presyn_spike_times_list))
    h.run()
    if i == 0:
        t_array = np.array(t)
    soma_voltage_array = np.array(soma_voltage)
    raw_soma_voltage_list.append(soma_voltage_array)
    t_offset += ISI
## remember to reset all the stimuli:
for i in range(20):
    vs_list[i].play(h.Vector())

But these are raw voltage recordings. Let's offset and truncate them for comparison to the expected traces:

In [ ]:
actual_compound_EPSP_list, _ = offset_and_truncate_trace_list(raw_soma_voltage_list, t_array, start=250.)

### Let's compare expected vs. actual traces:

In [ ]:
fig, axes = plt.subplots(1, 2, sharey=True, figsize=(6., 5.))
for i in range(20):
    axes[0].plot(truncated_t, expected_compound_EPSP_list[i])
    axes[1].plot(truncated_t, actual_compound_EPSP_list[i])
axes[0].set_xlabel('Time (ms)')
axes[1].set_xlabel('Time (ms)')
axes[0].set_ylabel('Expected EPSP Amplitude (mV)')
axes[0].set_title('Expected (Linear sum)')
axes[1].set_ylabel('Actual EPSP Amplitude (mV)')
axes[1].set_title('Actual')
fig.tight_layout()
fig.show()

### Now the range of depolarization is starting to reduce driving force on AMPA-Rs, so we're observing sublinear integration:

Let's compare the amplitudes directly:

In [ ]:
expected_EPSP_amplitude_list = []
actual_EPSP_amplitude_list = []
for i in range(20):
    this_expected_EPSP_amp = get_event_peak_amp(truncated_t, expected_compound_EPSP_list[i], baseline_window=(-10., 0.), \
                                                measurement_window=(0., 200.), direction=1.)
    expected_EPSP_amplitude_list.append(this_expected_EPSP_amp)
    this_actual_EPSP_amp = get_event_peak_amp(truncated_t, actual_compound_EPSP_list[i], baseline_window=(-10., 0.), \
                                                measurement_window=(0., 200.), direction=1.)
    actual_EPSP_amplitude_list.append(this_actual_EPSP_amp)

In [ ]:
max_amp = 1.1 * max(max(expected_EPSP_amplitude_list), max(actual_EPSP_amplitude_list))
plt.figure()
plt.scatter(expected_EPSP_amplitude_list, actual_EPSP_amplitude_list, c='k')
plt.plot((0., max_amp), (0., max_amp), '--', c='darkgrey')
plt.xlabel('Expected EPSP Amplitude (mV)')
plt.ylabel('Actual EPSP Amplitude (mV)')

## 6. Adding NMDA-type glutamate receptors to acheive supralinear input summation.

Based on the Popescu et al., 2004 paper, NMDA-Rs are expected to facilitate in response to repeated activation for two reasons: <br>
1. Mg2+ block will be partially alleviated with increased depolarization after the first pulse.
2. NMDA-Rs are not saturated by glutamate on the first pulse, and so more channels can bind and open on a 2nd pulse.

I wrote a point process that emulates these characteristic features of NMDA-Rs: `FacilNMDA` <br>
Let's take the first AMPA-type synapse that we had created, and we'll create a new `FacilNMDA` point process, and a new `NetCon` object, but we can actually share a `VecStim` with the AMPA synapse.

In [ ]:
AMPA_syn = syn_list[0]
vs = vs_list[0]
NMDA_syn = h.FacilNMDA(dend(0.75))
NMDA_nc = h.NetCon(vs, NMDA_syn)
NMDA_nc.delay = 0.
NMDA_nc.weight[0] = 1.
# This mechanism also uses the second element of the NetCon weight vector to specify the unitary single-pulse peak conductance
NMDA_g_unit = 0.0025
NMDA_nc.weight[1] = NMDA_g_unit

AMPA_current = h.Vector()
AMPA_current.record(AMPA_syn._ref_i, h.dt)
NMDA_current = h.Vector()
NMDA_current.record(NMDA_syn._ref_i, h.dt)

### Let's stimulate both AMPA and NMDA with a single pulse:

In [ ]:
presyn_spike_times_list = [250.]
vs.play(h.Vector(presyn_spike_times_list))
h.run()
vs.play(h.Vector())

In [ ]:
fig, axes = plt.subplots(2)
axes[0].plot(t, soma_voltage, label='Soma Vm')
axes[0].plot(t, dend_voltage, label='Dend Vm')
axes[1].plot(t, AMPA_current, label='AMPA')
axes[1].plot(t, NMDA_current, label='NMDA')
axes[0].set_ylabel('Voltage (mV)')
axes[1].set_ylabel('Current (nA)')
axes[0].set_xlabel('Time (ms)')
axes[1].set_xlabel('Time (ms)')
axes[0].legend(loc='best', frameon=False, framealpha=0.5)
axes[1].legend(loc='best', frameon=False, framealpha=0.5)
fig.tight_layout()
fig.show()

### How about a train?

In [ ]:
presyn_spike_times_list = [250. + 10. * i for i in range(5)]
vs.play(h.Vector(presyn_spike_times_list))
h.run()
vs.play(h.Vector())

In [ ]:
fig, axes = plt.subplots(2)
axes[0].plot(t, soma_voltage, label='Soma Vm')
axes[0].plot(t, dend_voltage, label='Dend Vm')
axes[1].plot(t, AMPA_current, label='AMPA')
axes[1].plot(t, NMDA_current, label='NMDA')
axes[0].set_ylabel('Voltage (mV)')
axes[1].set_ylabel('Current (nA)')
axes[0].set_xlabel('Time (ms)')
axes[1].set_xlabel('Time (ms)')
axes[0].legend(loc='best', frameon=False, framealpha=0.5)
axes[1].legend(loc='best', frameon=False, framealpha=0.5)
fig.tight_layout()
fig.show()

### Now let's put NMDA-Rs at all of our 20 synaptic sites:

In [ ]:
NMDA_syn_list = []
NMDA_nc_list = []
for i in range(20):
    vs = vs_list[i]
    NMDA_syn = h.FacilNMDA(dend(0.75))
    NMDA_syn_list.append(NMDA_syn)
    
    NMDA_nc = h.NetCon(vs, NMDA_syn)
    NMDA_nc.delay = 0.
    NMDA_nc.weight[0] = 1.
    # This mechanism also uses the second element of the NetCon weight vector to specify the unitary single-pulse peak conductance
    NMDA_nc.weight[1] = NMDA_g_unit
    NMDA_nc_list.append(NMDA_nc)

### Now let's repeat our synaptic summation experiment:

In [ ]:
presyn_spike_times_list = [250.]
raw_soma_unitary_EPSP_list = []
for i in range(20):
    vs_list[i].play(h.Vector(presyn_spike_times_list))
    h.run()
    vs_list[i].play(h.Vector())
    # save an array of the time base
    if i == 0:
        t_array = np.array(t)
    soma_voltage_array = np.array(soma_voltage)
    raw_soma_unitary_EPSP_list.append(soma_voltage_array)
    vs_list[i].play(h.Vector())

In [ ]:
# First let's baseline subtract and truncate these traces so that we can use them for offline summing:
truncated_soma_unitary_EPSP_list, truncated_t = offset_and_truncate_trace_list(raw_soma_unitary_EPSP_list, t_array, start=250.)

# Now we will build a list of traces with increasing numbers of units summed:
expected_compound_EPSP_list = get_expected_compound_EPSP_list(truncated_soma_unitary_EPSP_list, truncated_t, ISI=0.3)

In [ ]:
raw_soma_voltage_list = []

ISI = 0.3
t_offset = 0.
for i in range(20):
    presyn_spike_times_list = [250. + i * ISI]
    vs_list[i].play(h.Vector(presyn_spike_times_list))
    h.run()
    if i == 0:
        t_array = np.array(t)
    soma_voltage_array = np.array(soma_voltage)
    raw_soma_voltage_list.append(soma_voltage_array)
    t_offset += ISI
## remember to reset all the stimuli:
for i in range(20):
    vs_list[i].play(h.Vector())

In [ ]:
actual_compound_EPSP_list, _ = offset_and_truncate_trace_list(raw_soma_voltage_list, t_array, start=250.)

In [ ]:
fig, axes = plt.subplots(1, 2, sharey=True, figsize=(6., 5.))
for i in range(20):
    axes[0].plot(truncated_t, expected_compound_EPSP_list[i])
    axes[1].plot(truncated_t, actual_compound_EPSP_list[i])
axes[0].set_xlabel('Time (ms)')
axes[1].set_xlabel('Time (ms)')
axes[0].set_ylabel('Expected EPSP Amplitude (mV)')
axes[0].set_title('Expected (Linear sum)')
axes[1].set_ylabel('Actual EPSP Amplitude (mV)')
axes[1].set_title('Actual')
fig.tight_layout()
fig.show()

In [ ]:
expected_EPSP_amplitude_list = []
actual_EPSP_amplitude_list = []
for i in range(20):
    this_expected_EPSP_amp = get_event_peak_amp(truncated_t, expected_compound_EPSP_list[i], baseline_window=(-10., 0.), \
                                                measurement_window=(0., 200.), direction=1.)
    expected_EPSP_amplitude_list.append(this_expected_EPSP_amp)
    this_actual_EPSP_amp = get_event_peak_amp(truncated_t, actual_compound_EPSP_list[i], baseline_window=(-10., 0.), \
                                                measurement_window=(0., 200.), direction=1.)
    actual_EPSP_amplitude_list.append(this_actual_EPSP_amp)

In [ ]:
max_amp = 1.1 * max(max(expected_EPSP_amplitude_list), max(actual_EPSP_amplitude_list))
plt.figure()
plt.scatter(expected_EPSP_amplitude_list, actual_EPSP_amplitude_list, c='k')
plt.plot((0., max_amp), (0., max_amp), '--', c='darkgrey')
plt.xlabel('Expected EPSP Amplitude (mV)')
plt.ylabel('Actual EPSP Amplitude (mV)')

### Supralinear summation!

## 7. Papers for next week focusing on heterogeneity of interneuron cell types and their impact on dendritic integration.
- Pouille, F., Scanziani, M. Routing of spike series by dynamic circuits in the hippocampus. Nature 429, 717–723 (2004). https://doi.org/10.1038/nature02615
- Milstein, A., Bloss, E., Apostolides, P., Vaidya, S., Dilly, G., Zemelman, B., Magee, J. (2015). Inhibitory Gating of Input Comparison in the CA1 Microcircuit Neuron  87(6), 1274-1289. https://dx.doi.org/10.1016/j.neuron.2015.08.025